In [2]:
from mmnl import MMNL
import numpy as np
import pickle
import pandas as pd
import time

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Authorize Google Drive to upload files to the drive

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

NameError: name 'auth' is not defined

In [3]:
def load_data(path):
    dat = np.load(path)
    X = dat[:, :-1]
    Y = np.reshape(dat[:, -1], (-1, 1))
    return X, Y
X, Y = load_data('data/data.npy')

Load data

In [3]:
%%time
infile = open('./data/1000_Y_dgp.p', 'rb')
Y_dgp = pickle.load(infile)
# Y_dgp = big_dict['theta: [ 1.5  1.  -1.1  0.4  0.1  0.6]']
print(Y_dgp.shape, '\n',Y_dgp)

(2798, 1000) 
 [[3. 3. 0. ... 0. 1. 2.]
 [3. 3. 1. ... 0. 1. 3.]
 [1. 1. 1. ... 0. 3. 1.]
 ...
 [3. 3. 3. ... 3. 3. 3.]
 [3. 3. 3. ... 3. 3. 1.]
 [2. 2. 1. ... 2. 1. 2.]]
Wall time: 49.5 ms


In [12]:
%%time
m = [50,75,100,125]
for draws in m:
    t=time.time()
    result = []
    start = 0
    end = 200
    method = 'QMC'
    skips = 0
    for i in range(start,end):
        print('iteration: %i' %(i+1))
        try:
            qmc = MMNL(X,Y_dgp[:,i],draws,3,method=method)
            solution = qmc.solver()
            result.append(solution)
        except np.linalg.LinAlgError:
            skips += 1
            print('deze is overgeslagen, optimizer doet naar, %i skips'%(skips))
        if len(result)== 10:
            pickle.dump( result, open( "./resultaten/%s(%i)/%i-%i_%s(%i)_dgptest_results.p" %(method,draws,i-9,i,method,draws), "wb" ) )
            result = []
            skips = 0
    print('%s(%i) run time: %f'%(method,draws,time.time()-t))
#     f = drive.CreateFile()
#     f.SetContentFile("./resultaten/%i-%i_%s(%i)_results.p" %(i,i+10,method,draws))
#     f.Upload()

iteration: 1
      fun: 2929.3913198470946
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.01659828, -0.03096829, -0.04811227,  0.00013642,  0.        ,
        0.02278284])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 322
      nit: 34
   status: 0
  success: True
        x: array([  1.40621837,   1.06036242,  -1.04940274,  -8.7989295 ,
       -13.94105004,  -0.45934851]) 
 15.289263248443604
iteration: 2
      fun: 2883.465724793933
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00113687, -0.00186446, -0.00222826, -0.0005457 , -0.0001819 ,
       -0.00118234])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 154
      nit: 18
   status: 0
  success: True
        x: array([ 1.54983125,  0.81558495, -1.19100525, -0.53986228, -0.48371554,
       -0.54952656]) 
 7.253625392913818
iteration: 3
      fun: 2865.8470000993657
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
 